In [1]:
from langchain_openai import ChatOpenAI
import httpx

# Create an HTTP client that skips SSL verification (only for hackathon/test environments)
client = httpx.Client(verify=False)
llm = ChatOpenAI(
 base_url="https://genailab.tcs.in",
 model="azure/genailab-maas-gpt-4o",
 api_key="sk-K5cCB7yk7rc3ex2NV3-TWw",
 http_client=client
)

In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(
 base_url="https://genailab.tcs.in",
 model="azure/genailab-maas-text-embedding-3-large",
 api_key="sk-K5cCB7yk7rc3ex2NV3-TWw",
 http_client=client)

In [3]:
import requests
for method in ("get","post","put","delete","head","options","patch"):
    original = getattr(requests,method)

    def insecure_request(*args, _original = original, **kwargs):
        kwargs["verify"] = False
        return _original(*args,**kwargs)
    
    setattr(requests,method,insecure_request)

In [4]:
import os
print(os.getcwd())

c:\Users\genaiblrancusr103\Desktop\TEST


WIHTOUT LLM 

In [ ]:
# import os
# import pandas as pd
# from dotenv import load_dotenv
# from langchain_core.documents import Document
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from langchain_community.vectorstores import Chroma

# # --- Load CSV ---
# csv_path = r"c:\Users\genaiblrancusr103\Desktop\TEST\csvfiles\Dairy_Historical_Sales.csv"
# df = pd.read_csv(csv_path)
# df["Month"] = df["Month"].astype(str)

# # --- Create row-level Documents ---
# row_docs = []
# for _, row in df.iterrows():
#     page = (
#         f"Month: {row['Month']}, Milk: {row['Milk']}, Ghee: {row['Ghee']}, "
#         f"Curd: {row['Curd']}, Paneer: {row['Paneer']}, Cheese: {row['Cheese']}, "
#         f"Sweets: {row['Sweets']}"
#     )
#     row_docs.append(
#         Document(
#             page_content=page,
#             metadata={
#                 "month": row["Month"],
#                 "year": row["Month"][:4],
#                 "Milk": int(row["Milk"]),
#                 "Ghee": int(row["Ghee"]),
#                 "Curd": int(row["Curd"]),
#                 "Paneer": int(row["Paneer"]),
#                 "Cheese": int(row["Cheese"]),
#                 "Sweets": int(row["Sweets"]),
#             },
#         )
#     )

# # --- Create yearly chunks ---
# chunk_docs = []
# for year, group in df.groupby(df["Month"].str[:4]):
#     content_lines = [f"Year: {year}", "Month Milk Ghee Curd Paneer Cheese Sweets"]
#     for _, r in group.iterrows():
#         content_lines.append(
#             f"{r['Month']} {r['Milk']} {r['Ghee']} {r['Curd']} {r['Paneer']} {r['Cheese']} {r['Sweets']}"
#         )
#     chunk_docs.append(
#         Document(page_content="\n".join(content_lines), metadata={"year": year, "type": "year_chunk"})
#     )

# # --- Merge row-level + yearly chunks ---
# all_docs = row_docs + chunk_docs

# # --- Store in Chroma vector DB ---
# persist_dir = "chroma_db_dairy"
# vector_store = Chroma.from_documents(documents=all_docs, embedding=embedding_model, persist_directory=persist_dir)
# vector_store.persist()

# retriever = vector_store.as_retriever(search_kwargs={"k": 8})

# # --- Smart query handler ---
# def smart_query(query: str):
#     print(f"\n--- Query: {query} ---")

#     months_map = {
#         "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04", "May": "05", "Jun": "06",
#         "Jul": "07", "Aug": "08", "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"
#     }

#     tokens = query.split()
#     year = None
#     month_num = None
#     product = None

#     # crude parse: look for year, month, product
#     for t in tokens:
#         if t.isdigit() and len(t) == 4:
#             year = t
#         if t[:3].capitalize() in months_map:
#             month_num = months_map[t[:3].capitalize()]
#         if t.capitalize() in ["Milk", "Ghee", "Curd", "Paneer", "Cheese", "Sweets"]:
#             product = t.capitalize()

#     # Case 1: month + product + year → exact lookup
#     if year and month_num and product:
#         target_month = f"{year}-{month_num}"
#         exact = [doc for doc in row_docs if doc.metadata["month"] == target_month]
#         if exact:
#             print(f"{product} sales in {target_month}: {exact[0].metadata[product]}")
#             return

#     # Case 2: product + year (no month) → aggregate
#     if year and product and not month_num:
#         year_docs = [doc for doc in row_docs if doc.metadata["year"] == year]
#         if year_docs:
#             total = sum(doc.metadata[product] for doc in year_docs)
#             breakdown = "\n".join([f"{d.metadata['month']} → {d.metadata[product]}" for d in year_docs])
#             print(f"{product} sales in {year}:\n{breakdown}\nTotal {product} sales in {year}: {total}")
#             return

#     # Case 3: fallback → use retriever + LLM summarization
#     results = retriever.invoke(query)
#     context = "\n".join([d.page_content for d in results])
#     prompt = f"Answer the question based on the data:\n\n{context}\n\nQuestion: {query}"
#     answer = llm.invoke(prompt)




WITH LLM

In [137]:
# --- Example Queries ---
queries = [
    "Milk sales in Jan 2025",
    "Ghee sales in Feb 2014",
    "Ghee sales in 2025",
    "Summarize Cheese sales in 2025",
    "All sales in 2025"
]

for q in queries:
    print(f"\n--- Query: {q} ---")
    result = rag_chain.invoke({"input": q})
    print(result["answer"])


messages=[SystemMessage(content='You are a helpful assistant that answers questions based on retrieved documents.', additional_kwargs={}, response_metadata={}), HumanMessage(content="Answer the question using the following context:\n\n[Document(metadata={'type': 'year_chunk', 'year': '2025'}, page_content='Year: 2025\\nMonth Milk Ghee Curd Paneer Cheese Sweets\\n2025-01 717 143 215 71 57 107\\n2025-02 696 139 209 69 56 104\\n2025-03 775 155 233 77 62 116\\n2025-04 796 159 239 79 64 119\\n2025-05 858 171 257 85 69 129\\n2025-06 826 166 248 82 66 124\\n2025-07 783 157 235 78 63 118\\n2025-08 740 148 221 74 59 111\\n2025-09 702 140 210 70 56 105\\n2025-10 706 141 212 71 57 106'), Document(metadata={'year': '2025', 'type': 'year_chunk'}, page_content='Year: 2025\\nMonth Milk Ghee Curd Paneer Cheese Sweets\\n2025-01 717 143 215 71 57 107\\n2025-02 696 139 209 69 56 104\\n2025-03 775 155 233 77 62 116\\n2025-04 796 159 239 79 64 119\\n2025-05 858 171 257 85 69 129\\n2025-06 826 166 248 82 66

In [81]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vector_store = Chroma.from_documents(
    all_docs,
    embedding_model,
    persist_directory="chroma_db"
)
vector_store.persist()

In [92]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableMap


retriever = vector_store.as_retriever(search_kwargs={"k": 3})

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that answers questions based on retrieved documents."),
    ("human", "Answer the question using the following context:\n\n{context}\n\nQuestion: {question}")
])


rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [91]:
# # query = "Given a prediction for Ghee for 2026 year. Whether I need to make more orders for upcoming month?"
# query = "Give of Cheese sales for entire 2024"
# response = rag_chain.invoke(query)
# print(response.content)

TypeError: Number of requested results 0, cannot be negative, or zero. in query.

In [ ]:
# import os
# from dotenv import load_dotenv
# from langchain_community.document_loaders import PyPDFLoader, CSVLoader
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.documents import Document
# from langchain_community.vectorstores import Chroma

# # --- Load multiple PDFs ---
# pdf_files = [
#     r"c:\Users\genaiblrancusr103\Desktop\TEST\pdffiles\Food_Beverages_Events_2025_2026.pdf",
#     r"c:\Users\genaiblrancusr103\Desktop\TEST\pdffiles\India_FMCG_outlook_2025.pdf",
#     r"c:\Users\genaiblrancusr103\Desktop\TEST\pdffiles\Market_news_on_FMCG.pdf",
#     r"c:\Users\genaiblrancusr103\Desktop\TEST\pdffiles\Retail_marketing_calendar_2026.pdf",
#     r"c:\Users\genaiblrancusr103\Desktop\TEST\pdffiles\Snipp_2025_Retail_Seasonal_Promotion_Marketing_Calendar.pdf"
# ]

# pdf_docs = []
# for file in pdf_files:
#     loader = PyPDFLoader(file)
#     pdf_docs.extend(loader.load())

# # --- Load CSV file (optional, if you want to combine) ---
# csv_loader = CSVLoader(file_path=r"c:\Users\genaiblrancusr103\Desktop\TEST\csvfiles\Dairy_Historical_Sales.csv")
# csv_docs = csv_loader.load()

# # --- Combine all sources ---
# all_docs = pdf_docs + csv_docs

# # Combine all text into one block
# full_text = "\n".join([doc.page_content for doc in all_docs])

# # Agentic chunking prompt
# prompt = ChatPromptTemplate.from_messages([
#     ("system",
#      "You are a helpful assistant that chunks documents into semantically meaningful sections for retrieval. "
#      "Apply different strategies depending on the document type:\n"
#      " - For CSV/tabular data: group related rows together by logical dimensions such as time periods (e.g., yearly or quarterly), "
#      "product categories, or other natural groupings. Preserve the tabular structure in each chunk.\n"
#      " - For PDF/textual documents: split content into coherent sections based on headings, subheadings, or semantic meaning. "
#      "Keep paragraphs or related sentences together so that each chunk represents a complete idea.\n"
#      "Ensure chunks are concise but contextually complete."),
#     ("human",
#      "Split the following content into semantically meaningful chunks according to its type (CSV vs PDF). "
#      "For CSVs, group rows logically (e.g., by year, product, or category). "
#      "For PDFs, group paragraphs or sections by semantic meaning. "
#      "Return each chunk as a numbered list.\n\n{document}")
# ])

# # Format prompt with combined content
# formatted_prompt = prompt.format_messages(document=full_text)

# # Invoke LLM for agentic chunking
# response = llm.invoke(formatted_prompt)

# # Parse response into chunks
# chunks = response.content.split("\n\n")  # crude split; refine if needed

# # Convert chunks into LangChain Document objects
# chunked_docs = [Document(page_content=chunk.strip()) for chunk in chunks if chunk.strip()]



# # Agentic chunking prompt
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful assistant that chunks tabular data into semantically meaningful sections for retrieval."),
#     ("human", "Split the following CSV content into semantically meaningful chunks. "
#               "Group related rows together (e.g., by time period, category, or logical grouping). "
#               "Return each chunk as a numbered list.\n\n{document}")
# ])



In [ ]:
# import os
# from dotenv import load_dotenv
# from langchain_community.document_loaders import CSVLoader
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.documents import Document
# from langchain_community.vectorstores import Chroma

# # Use CSVLoader instead of PyPDFLoader
# loader = CSVLoader(file_path=r"c:\Users\genaiblrancusr103\Desktop\TEST\csvfiles\Dairy_Historical_Sales.csv")
# docs = loader.load()

# # Combine all rows into one text block
# full_text = "\n".join([doc.page_content for doc in docs])

# prompt = ChatPromptTemplate.from_messages([
#     ("system",
#      "You are a helpful assistant that chunks documents into semantically meaningful sections for retrieval. "
#      "Apply different strategies depending on the document type:\n"
#      " - For CSV/tabular data: group related rows together by logical dimensions such as time periods (e.g., yearly or quarterly), "
#      "product categories, or other natural groupings. Preserve the tabular structure in each chunk.\n"
#      " - For PDF/textual documents: split content into coherent sections based on headings, subheadings, or semantic meaning. "
#      "Keep paragraphs or related sentences together so that each chunk represents a complete idea.\n"
#      "Ensure chunks are concise but contextually complete."),
#     ("human",
#      "Split the following content into semantically meaningful chunks according to its type (CSV vs PDF). "
#      "For CSVs, group rows logically (e.g., by year, product, or category). "
#      "For PDFs, group paragraphs or sections by semantic meaning. "
#      "Return each chunk as a numbered list.\n\n{document}")
# ])


# # Format prompt with document content
# formatted_prompt = prompt.format_messages(document=full_text)

# # Invoke LLM for agentic chunking
# response = llm.invoke(formatted_prompt)

# # Parse response into chunks
# chunks = response.content.split("\n\n")  # crude split; refine if needed

# # Convert chunks into LangChain Document objects
# chunked_docs = [Document(page_content=chunk.strip()) for chunk in chunks if chunk.strip()]



In [ ]:
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# vector_store = Chroma.from_documents(
#     chunked_docs,
#     embedding_model,
#     persist_directory="chroma_db"
# )
# vector_store.persist()

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.runnables import RunnableMap

# retriever = vector_store.as_retriever(search_kwargs={"k": 3})
# prompt = ChatPromptTemplate.from_messages([
#     ("system",
#      "You are a Sales Analyzer and forecasting assistant that predicts future monthly sales for dairy products.\n"
#      "You need to act as assistant according to the Query which is coming from the user. \n"
#      "Ground your work in:\n"
#      "1) Dairy historical sales CSV (columns: Month, Milk, Ghee, Curd, Paneer, Cheese, Sweets).\n"
#      "2) External market documents: Food_Beverages_Events_2025_2026.pdf, India_FMCG_outlook_2025.pdf, "
#      "Market_news_on_FMCG.pdf, Retail_marketing_calendar_2026.pdf, "
#      "Snipp_2025_Retail_Seasonal_Promotion_Marketing_Calendar.pdf.\n\n"
#      "Strict rules:\n"
#      "- Use the entire historical dataset for forecasting.\n"
#      "- If the question mentions one product, return only that product; if multiple, return product-wise.\n"
#      "- The output table must first list the last 10 rows of historical sales for each requested product, "
#      "with the Sales column containing the exact numeric values copied verbatim from the provided context. "
#      "Never replace numeric values with text like 'Historical Data'.\n"
#      "- After the 10 historical rows, append predicted rows for the requested months.\n"
#      "- Predictions must consider seasonal patterns, event calendars, promotions, and market news as Key Drivers.\n"
#      "- Highlight Risks/Uncertainty when present (e.g., supply chain issues, policy changes, macro slowdown).\n"
#      "- If the question specifies months/years, predict exactly those; otherwise predict the next 6 months beyond "
#      "the latest historical month when the question asks for 'next 6 months'.\n"
#      "- Date formats: Month = YYYY-MM; Year = 4-digit year taken from the Month.\n"
#      "- If a historical value is not present in the context for a given Month/Product, write 'Insufficient context' "
#      "in Sales for that row and explain in Key Drivers; do not invent numbers.\n"
#      "- Output must be a single table with these columns only:\n"
#      "  Month | Year | Product | Sales | Key Drivers | Risks/Uncertainty.\n"),
#     ("human",
#      "Using the following combined context (historical CSV + external market documents):\n\n{context}\n\n"
#      "Task:\n"
#      "1) Identify the products and months mentioned in the question.\n"
#      "2) Output the last 10 rows of historical sales for each requested product, copying the Sales values exactly from the context.\n"
#      "3) Append predicted rows for the requested months (e.g., 'next 6 months'), with numeric Sales values.\n"
#      "4) Provide concise Key Drivers (seasonality, events, promotions, market news) and note Risks/Uncertainty when applicable.\n\n"
#      "Return only the table with the exact columns specified, no prose.\n\n"
#      "Question: {question}")
# ])

# # Chain: retrieve → format prompt → invoke LLM
# rag_chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | prompt
#     | llm
# )

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.runnables import RunnableMap


# retriever = vector_store.as_retriever(search_kwargs={"k": 3})
# # prompt = ChatPromptTemplate.from_messages([
# #     ("system", 
# #      "You are a helpful assistant that predicts future sales based on historical sales data. "
# #      "Use the retrieved context (past monthly sales) to identify trends, seasonality, and patterns. "
# #      "Always return the output in a clear tabular format with columns for Month, Year, and Predicted Sales."),
# #     ("human", 
# #      "Using the following historical sales context:\n\n{context}\n\n"
# #      "Predict the sales for the months mentioned in the question. "
# #      "Return the results strictly in tabular format.\n\nQuestion: {question}")
# # ])
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful assistant that answers questions based on retrieved documents."),
#     ("human", "Answer the question using the following context:\n\n{context}\n\nQuestion: {question}")
# ])


# # Chain: retrieve → format prompt → invoke LLM
# rag_chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | prompt
#     | llm
# )

In [69]:
# # query = "Given a prediction for Ghee for 2026 year. Whether I need to make more orders for upcoming month?"
# query = "what are Ghee sales in July 2025"
# response = rag_chain.invoke(query)
# print(response.content)

The provided documents do not mention ghee sales for July 2025 specifically. The text emphasizes trends and patterns but lacks explicit monthly numerical data for ghee or other products for that specific time. For precise figures, direct access to the detailed monthly sales data would be necessary.


In [ ]:
# from langchain_openai import ChatOpenAI
# import httpx

# # Create an HTTP client that skips SSL verification (only for hackathon/test environments)
# client = httpx.Client(verify=False)
# llm = ChatOpenAI(
#  base_url="https://genailab.tcs.in",
#  model="azure/genailab-maas-gpt-4o",
#  api_key="sk-A5iYc_U5ojeN6ZO4I_qvCQ",
#  http_client=client
# )
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# embedding_model = OpenAIEmbeddings(
#  base_url="https://genailab.tcs.in",
#  model="azure/genailab-maas-text-embedding-3-large",
#  api_key="sk-A5iYc_U5ojeN6ZO4I_qvCQ",
#  http_client=client)
# import os
# print(os.getcwd())
# import requests
# for method in ("get","post","put","delete","head","options","patch"):
#     original = getattr(requests,method)

#     def insecure_request(*args, _original = original, **kwargs):
#         kwargs["verify"] = False
#         return _original(*args,**kwargs)
    
#     setattr(requests,method,insecure_request)
# import os
# from dotenv import load_dotenv
# from langchain_community.document_loaders import PyPDFLoader
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.messages import HumanMessage
# from langchain_core.documents import Document
# from langchain_community.vectorstores import Chroma

# loader = PyPDFLoader(r"c:\Users\Genaiblrpiousr13\Downloads\test\checking\files\photos.pdf")
# docs = loader.load()
# full_text = "\n".join([doc.page_content for doc in docs])

# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful assistant that chunks documents for semantic search."),
#     ("human", "Split the following document into semantically meaningful sections. Return each chunk as a numbered list.\n\n{document}")
# ])

# formatted_prompt = prompt.format_messages(document=full_text)

# response = llm.invoke(formatted_prompt)
# chunks = response.content.split("\n\n")  # crude split; refine if needed

# chunked_docs = [Document(page_content=chunk.strip()) for chunk in chunks if chunk.strip()]
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# vector_store = Chroma.from_documents(
#     chunked_docs,
#     embedding_model,
#     persist_directory="chroma_db"
# )
# vector_store.persist()
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.runnables import RunnableMap


# retriever = vector_store.as_retriever(search_kwargs={"k": 3})
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful assistant that answers questions based on retrieved documents."),
#     ("human", "Answer the question using the following context:\n\n{context}\n\nQuestion: {question}")
# ])

# # Chain: retrieve → format prompt → invoke LLM
# rag_chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | prompt
#     | llm
# )
# query = "Explain the Camera Handling skills"
# response = rag_chain.invoke(query)
# print(response.content)

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate

# prompt = ChatPromptTemplate.from_messages([
#     ("system",
#      "You are a forecasting assistant that predicts future monthly sales for dairy products based on:\n"
#      "1) Dairy historical sales CSV with columns: Month, Milk, Ghee, Curd, Paneer, Cheese, Sweets.\n"
#      "2) External market documents: Food_Beverages_Events_2025_2026.pdf, India_FMCG_outlook_2025.pdf, "
#      "Market_news_on_FMCG.pdf, Retail_marketing_calendar_2026.pdf, "
#      "Snipp_2025_Retail_Seasonal_Promotion_Marketing_Calendar.pdf, Dairy_Historical_Sales.csv.\n\n"
#      "Core rules:\n"
#      "- Use retrieved context only. Do not invent data.\n"
#      "- Respect the requested products: if the question mentions one product, return only that product; if multiple, return each product separately.\n"
#      "- For each product predicted, include the last 10 months of its historical sales explicitly in the output.\n"
#      "- Incorporate seasonal patterns, event calendars, promotions, and market news as key drivers.\n"
#      "- Flag risks or uncertainty (e.g., supply chain issues, policy changes, macro slowdown) when present in the context.\n"
#      "- If the question specifies months/years, predict for exactly those. If not, predict for the next 3 future months beyond the latest historical date.\n"
#      "- Handle date parsing robustly: Month is YYYY-MM, Year is the 4-digit year.\n"
#      "- If any required context for a product/month is missing, state 'Insufficient context' for Predicted Demand Sales and explain in Key Drivers.\n"
#      "- Output must be strictly tabular with these columns and no extra commentary:\n"
#      "  Month | Year | Product | Last 10 Months Sales | Predicted Demand Sales | Key Drivers | Risks/Uncertainty.\n"),
#     ("human",
#      "Using the following combined context (historical CSV + external market documents):\n\n{context}\n\n"
#      "Task:\n"
#      "1) Identify the products and months mentioned in the question.\n"
#      "2) For each product, list the last 10 months of historical sales from the CSV.\n"
#      "3) Predict the sales for the requested months using historical trends and external drivers.\n"
#      "4) Provide concise Key Drivers (seasonality, events, promotions, market news) and note Risks/Uncertainty when applicable.\n\n"
#      "Return only a table with the exact columns specified, no prose.\n\n"
#      "Question: {question}")
# ])


In [ ]:
# prompt = ChatPromptTemplate.from_messages([
#     ("system", 
#      "You are a helpful assistant that predicts future sales for one or more products "
#      "based on Dairy historical sales data and supporting market documents. "
#      "You must use the retrieved context from multiple sources "
#      "(Dairy historical sales CSV with products: Milk, Ghee, Curd, Paneer, Cheese, Sweets; "
#      "1. Food_Beverages_Events_2025_2026.pdf 2. India_FMCG_outlook_2025.pdf 3. Market_news_on_FMCG.pdf 4. Retail_marketing_calendar_2026.pdf 5. Snipp_2025_Retail_Seasonal_Promotion_Marketing_Calendar.pdf 6. Dairy_Historical_Sales.csv) "
#      "to identify trends, seasonality, promotions, and external factors that may impact sales. "
#      "When a prediction is requested:\n"
#      " - If the question mentions a single product, return predictions only for that product from Dairy Historical sales CSV.\n"
#      " - If the question mentions multiple products, return predictions product‑wise from Dairy Historical sales CSV.\n"
#      " - A short explanation mentioning seasonal effects, holidays and market news from the supporting market document(1. Food_Beverages_Events_2025_2026.pdf 2. India_FMCG_outlook_2025.pdf 3. Market_news_on_FMCG.pdf 4. Retail_marketing_calendar_2026.pdf 5. Snipp_2025_Retail_Seasonal_Promotion_Marketing_Calendar.pdf 6. Dairy_Historical_Sales.csv).\n"
#      " - Highlight if any risks or uncertainty to watch for as an alert.\n"
#      " - Always include the last 10 months of historical sales and continue to provide the predicted data of given months in the query.\n"
#      " - Always return the output in a clear tabular format with columns:\n"
#      "   Month | Year | Product | Predicted Demand Sales | Key Drivers."),
#     ("human", 
#      "Using the following context from all available documents:\n\n{context}\n\n"
#      "Predict the sales for the product(s) and months mentioned in the question. "
#      "Return the results strictly in tabular format, including the last 10 months of sales "
#      "and key drivers for each prediction.\n\n"
#      "Question: {question}")
# ])